## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://python.langchain.com/v0.2/docs/versions/v0_2/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [3]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai langchain-qdrant

We'll also get the "star of the show" today, which is Ragas!

In [4]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [5]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [29]:
import os
import openai
from getpass import getpass

# openai.api_key = getpass("Please provide your OpenAI Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["LANGCHAIN_TRACING_V2"] = "false"

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.2.0

Building on what we've been learning, we'll be leveraging LangChain v0.2.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

- [`PyMuPDFLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [7]:
from langchain_community.document_loaders import PyMuPDFLoader

# PDF_LINK = "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf"
PDF_LINK = "PRECISE_StudyProtocol.pdf"
loader = PyMuPDFLoader(
    PDF_LINK  ### INSERT CODE
)

documents = loader.load()### YOUR CODE HERE

In [8]:
documents[0].metadata

{'source': 'PRECISE_StudyProtocol.pdf',
 'file_path': 'PRECISE_StudyProtocol.pdf',
 'page': 0,
 'total_pages': 76,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.24',
 'creationDate': 'D:20230616195132Z',
 'modDate': 'D:20230616195132Z',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain-text-splitters-character-recursivecharactertextsplitter)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 1500
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
documents = text_splitter.split_documents(documents)  

Let's confirm we've split our document.

In [10]:
len(documents)

163

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

- [`OpenAIEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html#langchain-openai-embeddings-base-openaiembeddings)

> NOTE: We are purposefully using an older embedding model to try and answer the guiding question: Is TE3 better than Ada-002?

In [11]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(
    model = EMBEDDING_MODEL  ### YOUR CODE HERE
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

- [`Qdrant`](https://api.python.langchain.com/en/latest/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore)

> NOTE: You'll need to provide the embedding dimension for Ada-002!

In [17]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# LOCATION = ":memory:"
# I HAVE ALREADY CREATED A COLLECTION WITH 200 AND 500 CHUNKS AND THESE ARE ALREADY IN QDRANT
LOCATION = "localhost:55001"
COLLECTION_NAME = "PRECISE-1500-50-ADA"
VECTOR_SIZE = 1536

In [20]:
qdrant_client = QdrantClient(LOCATION)  ### YOUR CODE HERE

# qdrant_client.create_collection(
#     collection_name=COLLECTION_NAME,
#     vectors_config=VectorParams(size=VECTOR_SIZE,distance=Distance.COSINE)
    
    ### YOUR CODE HERE
# )

In [14]:


# qdrant_vector_store = QdrantVectorStore(
#     client=qdrant_client,
#     collection_name=COLLECTION_NAME,
#     embedding=embeddings



#     ### YOUR CODE HERE
# )

# qdrant_vector_store.add_documents(documents)

In [21]:
qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="PRECISE-200-50-ADA",
    embedding=embeddings
)
retriever_200_50 = qdrant_vector_store.as_retriever(search_kwargs={"k":100})
# retriever = vectorstore.as_retriever(kwargs={"k":10})
retrieved_documents = retriever_200_50.invoke("WHat are the inclusion criteria for PRECISE?")
print(len(retrieved_documents))
# for doc in retrieved_documents:
#   print(doc)

100


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=ac47a0d3-d85e-4ce8-8580-f97789e4ea02,id=ac47a0d3-d85e-4ce8-8580-f97789e4ea02
Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=0ab180d3-cdfc-43a6-88af-09d079e619b0,id=0ab180d3-cdfc-43a6-88af-09d079e619b0; trace=41043ac5-6845-4cbf-9824-c2b2334f65f5,id=41043ac5-6845-4cbf-9824-c2b2334f65f5; trace=2ef5203a-9e8e-4656-a799-46a36e87e7c9,id=2ef5203a-9e8e-4656-a799-46a36e87e7c9; trace=2ef5203a-9e8e-4656-a799-46a36e87e7c9,id=c4ead6ec-cc

In [22]:
qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="PRECISE-500-50-ADA",
    embedding=embeddings
)
retriever_500_50 = qdrant_vector_store.as_retriever(search_kwargs={"k":25})
retrieved_documents = retriever_500_50.invoke("WHat are the inclusion criteria for PRECISE?")
# for doc in retrieved_documents:
#   print(doc)

In [23]:
qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="PRECISE-1500-50-ADA",
    embedding=embeddings
)
retriever_1500_50 = qdrant_vector_store.as_retriever(search_kwargs={"k":10})
retrieved_documents = retriever_1500_50.invoke("WHat are the inclusion criteria for PRECISE?")
# for doc in retrieved_documents:
#   print(doc)

In [86]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a helpful assistant who is an expert on clinical trials.  Answer the question based on information in the context.  If you cannot answer the
question then say you do not know.

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)   ### YOUR CODE HERE

#### Setting Up our Basic QA Chain

I am going to set up three RAG chains one for each retriever.

In [41]:
from operator import itemgetter

from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain_200 = (
    {"context": itemgetter("question") | retriever_200_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)
retrieval_augmented_qa_chain_500 = (
    {"context": itemgetter("question") | retriever_500_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)
retrieval_augmented_qa_chain_1500 = (
    {"context": itemgetter("question") | retriever_1500_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

local_llm = OpenAI(model= "TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q8_0.gguf",
                   base_url="http://192.168.1.239:1234/v1", api_key="not needed")

retrieval_augmented_qa_chain_1500_LOCAL = (
    {"context": itemgetter("question") | retriever_1500_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | local_llm, "context": itemgetter("context")}
)

In [42]:
question = "What are the inclusion and exclusion criteria for PRECISE?"

result = retrieval_augmented_qa_chain_1500_LOCAL.invoke({"question" : question})
print("Chunk size 150, LOCAL MODEL\n")
print(result["response"].content)
print("--------------\n")

BadRequestError: Error code: 400 - {'error': '<LM Studio error> . Error Data: n/a, Additional Data: n/a'}

In [44]:
import requests

url = "http://192.168.1.239:1234/v1/completions"
headers = {
    "Content-Type": "application/json"
    # "Authorization": f"Bearer {api_key}"  # or omit if not needed
}
data = {
    "model": "TheBloke/dolphin-2.7-mixtral-8x7b.Q8_0.gguf",
    "prompt": "What are the inclusion and exclusion criteria for PRECISE?",
    "max_tokens": 150
}

response = requests.post(url, headers=headers, json=data)
print(response.json())


{'id': 'cmpl-l4w883qf9hyyiso6trajj', 'object': 'text_completion', 'created': 1725832928, 'model': 'TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/dolphin-2.7-mixtral-8x7b.Q8_0.gguf', 'choices': [{'index': 0, 'text': '\n\nInclusion Criteria:\n1. The patient must be 18 years of age or older at the time of consent.\n2. The patient must have a clinical diagnosis of Parkinson\'s disease (PD).\n3. The patient must have an Hoehn & Yahr stage of 1-4 during the ON state.\n4. The patient must be taking a stable dose of levodopa and/or dopamine agonists for at least four weeks prior to screening, and they should continue on this regimen throughout the entire study period.\n5. The patient must have a minimum "Off" time of 2 hours per day as documented in their diary.\n6. The patient must', 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 15, 'completion_tokens': 149, 'total_tokens': 164}}


In [70]:
from langchain.llms.base import BaseLLM
from langchain.schema import LLMResult, Generation
from typing import Optional, List
import requests
from pydantic import Field

class CustomLLM(BaseLLM):
    model_url: str = Field(...)
    model_name: str = Field(...)

    def __init__(self, model_url: str, model_name: str, **kwargs):
        super().__init__(**kwargs)
        object.__setattr__(self, 'model_url', model_url)
        object.__setattr__(self, 'model_name', model_name)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
        }
        data = {
            "model": self.model_name,
            "prompt": prompt,
            # "max_tokens": 150,
        }
        response = requests.post(self.model_url, headers=headers, json=data)
        return response.json().get('choices', [{}])[0].get('text', '')

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        """This method must generate the output based on the prompts."""
        generations = []
        for prompt in prompts:
            response_text = self._call(prompt, stop)
            generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)

    @property
    def _identifying_params(self) -> dict:
        return {"model_url": self.model_url, "model_name": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom_llm"






In [79]:
dolphin_llm = CustomLLM(model_url="http://192.168.1.239:1234/v1/completions", 
                      model_name="TheBloke/dolphin-2.7-mixtral-8x7b.Q8_0.gguf")

llama_llm = CustomLLM(model_url="http://192.168.1.239:1234/v1/completions", 
                      model_name="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf")


In [80]:
retrieval_augmented_qa_chain_1500_DOLPHIN = (
    {"context": itemgetter("question") | retriever_1500_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | dolphin_llm, "context": itemgetter("context")}
)

retrieval_augmented_qa_chain_1500_LLAMA = (
    {"context": itemgetter("question") | retriever_1500_50, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llama_llm, "context": itemgetter("context")}
)

In [81]:
question = "What are the inclusion and exclusion criteria for PRECISE?"

result = retrieval_augmented_qa_chain_1500_DOLPHIN.invoke({"question" : question})
print("DOLPHIN Model\n")
print(result["response"])


DOLPHIN Model




In [83]:
result = retrieval_augmented_qa_chain_1500_LLAMA.invoke({"question" : question})
print("LLAMA Model\n")
print(result["response"])

KeyboardInterrupt: 

Let's test it out!

In [87]:
question = "What are the inclusion and exclusion criteria for PRECISE?"

result = retrieval_augmented_qa_chain_200.invoke({"question" : question})
print("Chunk size 200\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_500.invoke({"question" : question})
print("Chunk size 500\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_1500.invoke({"question" : question})
print("Chunk size 1500\n")
print(result["response"].content)
print("--------------\n")


Chunk size 200

The inclusion criteria for the PRECISE study are:
- Age between ≥40 weeks corrected gestational age to < 18 years; AND
- Admission to the PICU or CICU; AND
- Onset of ≥2 new organ dysfunctions within the last 3 calendar days (compared to pre-sepsis baseline) as measured by the modified Proulx criteria.

The exclusion criteria for the PRECISE study are:
- Weight <3kg; OR
- Limitation of care order at the time of screening; OR
- Known pregnancy; OR
- Lactating females; OR
- Receipt of anakinra or GM-CSF within the previous 28 days; OR
- Resolution of MODS by MODS Day 2; OR
- Previous enrollment in the PRECISE study.
--------------

Chunk size 500

The inclusion and exclusion criteria for the PRECISE study are as follows:

**Inclusion Criteria:**
- Participants must be aged ≥40 weeks corrected gestational age to < 18 years.
- Participants must have at least single organ failure on Study Day 1.

**Exclusion Criteria:**
- Weight <3 kg.
- Limitation of care order at the time 

In [28]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain_200.invoke({"question" : question})
print(result["response"].content)
print(result["context"])

result = retrieval_augmented_qa_chain_500.invoke({"question" : question})
print(result["response"].content)
print(result["context"])

result = retrieval_augmented_qa_chain_1500.invoke({"question" : question})
print(result["response"].content)
print(result["context"])

I do not know.
[Document(metadata={'source': 'PRECISE_StudyProtocol.pdf', 'file_path': 'PRECISE_StudyProtocol.pdf', 'page': 39, 'total_pages': 76, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.24', 'creationDate': 'D:20230616195132Z', 'modDate': 'D:20230616195132Z', 'trapped': '', '_id': '53e84942-5959-4f0a-ba46-3274988096db', '_collection_name': 'PRECISE-200-50-ADA'}, page_content='throughout the trial to preferentially allocate patients to arms appearing superior based on accu-\nmulating data.80 We will utilize the “Goldilocks” principle of frequent looks at the accumulating'), Document(metadata={'source': 'PRECISE_StudyProtocol.pdf', 'file_path': 'PRECISE_StudyProtocol.pdf', 'page': 56, 'total_pages': 76, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.24', 'creationDate': 'D:20230616195132Z', 'modDate':

In [92]:
question = "What are the exclusion criteria for PRECISE?"

result = retrieval_augmented_qa_chain_200.invoke({"question" : question})
print("Chunk size 200\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_500.invoke({"question" : question})
print("Chunk size 500\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_1500.invoke({"question" : question})
print("Chunk size 1500\n")
print(result["response"].content)
print("--------------\n")

Chunk size 200

The exclusion criteria for the PRECISE study are as follows:

- Weight <3 kg; OR
- Limitation of care order at the time of screening; OR
- Known pregnancy; OR
- Lactating females; OR
- Receipt of anakinra or GM-CSF within the previous 28 days; OR
- Resolution of MODS by MODS Day 2; OR
- Previous enrollment in the PRECISE study.
--------------

Chunk size 500

The exclusion criteria for the PRECISE study are as follows:

1. Weight <3kg
2. Limitation of care order at the time of screening
3. Peripheral white blood cell count < 1,000 cells/mm³ as a result of myeloablative therapy or receipt of myeloablative therapy within the previous 14 days
4. Known allergy to GM-CSF, anakinra, or E. coli-derived products
5. Dependence on maintenance of strict ketosis
6. Known pregnancy
7. Lactating females
8. Receipt of anakinra or GM-CSF within the previous 28 days
9. Resolution of MODS by MODS Day 2
10. Previous enrollment in the PRECISE study.
--------------

Chunk size 1500

The exc

In [94]:
question = "What are the inclusion criteria for PRECISE?"

result = retrieval_augmented_qa_chain_200.invoke({"question" : question})
print("Chunk size 200\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_500.invoke({"question" : question})
print("Chunk size 500\n")
print(result["response"].content)
print("--------------\n")

result = retrieval_augmented_qa_chain_1500.invoke({"question" : question})
print("Chunk size 1500\n")
print(result["response"].content)
print("--------------\n")

Chunk size 200

The inclusion criteria for the PRECISE study are as follows:

- Age: Participants must be ≥40 weeks corrected gestational age to < 18 years.
- Admission to the PICU or CICU.
- Onset of ≥2 new organ dysfunctions within the last 3 calendar days (compared to pre-sepsis baseline) as measured by the modified Proulx criteria.
--------------

Chunk size 500

The inclusion criteria for the PRECISE study are as follows:

- Participants must be aged ≥40 weeks corrected gestational age to < 18 years.
--------------

Chunk size 1500

The inclusion criteria for the PRECISE study are as follows:

1. Age: Participants must be ≥40 weeks corrected gestational age to < 18 years.
2. Admission: Participants must be admitted to the Pediatric Intensive Care Unit (PICU) or Cardiac Intensive Care Unit (CICU).
3. Organ Dysfunction: Participants must have the onset of ≥2 new organ dysfunctions within the last 3 calendar days (compared to pre-sepsis baseline) as measured by the modified Proulx cr

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [33]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [34]:
len(eval_documents)

624

> NOTE: 🛑 Running this cell as presented will incur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step. **YOU CAN SKIP THIS STEP BY LOADING THE `.csv` DIRECTLY FROM OUR REPOSITORY.** 🛑

#### Optional: SDG for Evaluation

In [35]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the four major forms of chance and th...,[that chance is immune from human intervention...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,Temporary subfolders in email management are u...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What impact does a severe personality disorder...,[added humor value.]\nWhile I enjoyed Marc’s p...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What is the relationship between precocious in...,[These three components are conspicuously link...,The context suggests that precocious individua...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some strategies for raising angel mon...,[This obviously raises the issue of how you’re...,The context suggests several strategies for ra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the Influence-from-Mere-Association Te...,[One very practical consequence of Liking/Lovi...,The Influence-from-Mere-Association Tendency r...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What should individuals focus on to succeed in...,[becomes irrelevant to determining the success...,"According to Dr. Simonton, individuals should ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the significance of being an individua...,[from scratch. This is a sharp diWerence from ...,Being an individual contributor is significant...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What makes engineering degrees considered usef...,[workforce in a high-impact way when you gradu...,Engineering degrees are considered useful in t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What services are people abandoning in favor o...,[billion people online now. That is a very lar...,People are abandoning older services like news...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


Let's look at the output and see what we can learn about it!

In [36]:
testset.test_data[0]

DataRow(question='What are the four major forms of chance and their distinct roles in human interactions?', contexts=['that chance is immune from human interventions. However, one\nmust be careful not to read any unconsciously purposeful intent\ninto “interventions”… [which] are to be viewed as accidental,\nunwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…\nWe can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:\nThe four kinds of chance each have a diWerent kind of motor'], ground_truth='The answer to given question is not present in context', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The

In [37]:
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv")

#### PREFERRED: Download `.csv` from DataRepository

I have to consume $85 before the end of September because credits I bought a year ago are going to expire!

In [ ]:
# !git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 87 (delta 24), reused 28 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (87/87), 70.09 MiB | 33.73 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
# !mv DataRepository/testset.csv .

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [38]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [39]:
test_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,What are the four major forms of chance and th...,['that chance is immune from human interventio...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,1,What is the purpose of temporary subfolders in...,['you can reply to a lot of messages with “I’m...,Temporary subfolders in email management are u...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,2,What impact does a severe personality disorder...,['added humor value.]\nWhile I enjoyed Marc’s ...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,3,What is the relationship between precocious in...,['These three components are conspicuously lin...,The context suggests that precocious individua...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,4,What are some strategies for raising angel mon...,['This obviously raises the issue of how you’r...,The context suggests several strategies for ra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,5,What is the Influence-from-Mere-Association Te...,['One very practical consequence of Liking/Lov...,The Influence-from-Mere-Association Tendency r...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,6,What should individuals focus on to succeed in...,['becomes irrelevant to determining the succes...,"According to Dr. Simonton, individuals should ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,7,What is the significance of being an individua...,['from scratch. This is a sharp diWerence from...,Being an individual contributor is significant...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,8,What makes engineering degrees considered usef...,['workforce in a high-impact way when you grad...,Engineering degrees are considered useful in t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,9,What services are people abandoning in favor o...,['billion people online now. That is a very la...,People are abandoning older services like news...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [40]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [41]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [42]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [43]:
response_dataset[0]

{'question': 'What are the four major forms of chance and their distinct roles in human interactions?',
 'answer': 'The four major forms of chance mentioned in the context are unwilled, inadvertent, and unforeseeable. Each of these forms plays distinct roles in human interactions, involving different kinds of exploratory activity and sensory receptivity. However, the specific details about the four forms of chance and their distinct roles are not provided in the context. Therefore, I cannot provide a complete answer regarding the four major forms of chance and their distinct roles.',
 'contexts': ['We can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:',
  'unwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…',
  'The four kinds of chance each ha

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [44]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [45]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [46]:
results

{'faithfulness': 0.6709, 'answer_relevancy': 0.7698, 'context_recall': 0.7500, 'context_precision': 0.6972, 'answer_correctness': 0.6469}

In [47]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are the four major forms of chance and th...,The four major forms of chance mentioned in th...,[We can observe chance arriving in four major ...,The answer to given question is not present in...,0.600000,0.000000,1.000000,0.000000,0.188577
1,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email m...,"[the normal course of your day.\nFourth, aside...",Temporary subfolders in email management are u...,0.500000,1.000000,1.000000,1.000000,0.994009
2,What impact does a severe personality disorder...,A severe personality disorder can lead a manag...,[severe personality disorder who micromanages ...,The answer to given question is not present in...,0.600000,1.000000,1.000000,0.000000,0.179351
3,What is the relationship between precocious in...,The relationship between precocious individual...,"[early, end late, and produce at above-average...",The context suggests that precocious individua...,0.857143,0.983589,1.000000,1.000000,0.765158
4,What are some strategies for raising angel mon...,Some strategies for raising angel money before...,[This obviously raises the issue of how you’re...,The context suggests several strategies for ra...,0.700000,1.000000,0.500000,0.805556,0.862922
5,What is the Influence-from-Mere-Association Te...,I do not know.,[The Psychology of Entrepreneurial Misjudgment...,The Influence-from-Mere-Association Tendency r...,0.000000,0.000000,0.000000,0.000000,0.182111
6,What should individuals focus on to succeed in...,"According to Dr. Simonton, individuals should ...",[progress through a creative career. Instead y...,"According to Dr. Simonton, individuals should ...",1.000000,0.981602,1.000000,1.000000,0.804426
7,What is the significance of being an individua...,The significance of being an individual contri...,[destiny — you get to succeed or fail on your ...,Being an individual contributor is significant...,0.444444,1.000000,0.666667,1.000000,0.843277
8,What makes engineering degrees considered usef...,Engineering degrees are considered useful in t...,[Which undergraduate degrees are useful in\nth...,Engineering degrees are considered useful in t...,0.857143,0.987121,1.000000,1.000000,0.539575
9,What services are people abandoning in favor o...,The context does not specify the exact service...,[billion people online now. That is a very lar...,People are abandoning older services like news...,0.750000,0.000000,1.000000,1.000000,0.407606


## Task : Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #1:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [48]:
te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [49]:
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME+"TE3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME+"TE3",
    embedding=te3_embeddings,
)

qdrant_vector_store.add_documents(documents)

['b47e0fc002b849518c4ccc41626165a9',
 'b40a34304ab54f279f4b513793cbfdee',
 'f72c0d8a660e4b7ba394aa495a161b06',
 'cbe4570cc6bb479597addfadb552c244',
 '0d2f7d76517d47449b7e366a8d8ff82c',
 '95cb90ea5486428e85461c0fc5e420c7',
 'eba9bffe8765446cad2edf9a557fef84',
 'e5921ac63e474bc7b4675789d89862a9',
 'ed4fa26402f84c4587b569a48a7b8438',
 '9e4d2c09965747e0bf3b7c9d310171b3',
 '593ac9ee1c3a4bdebbe552797b858efa',
 'fa13350c34b640119d3c1393de20ff3f',
 'dc4b549d0d164929a3da5066434178c0',
 'c7615c4dabc04306a6ac2440d0126ef3',
 '35d02a055b2d4780b77fbf58adc9758e',
 '970b7e63bcf44d75bb1c8e47dd51ec20',
 '3d5ee7c86c0542ea8ff03d96a57b827b',
 '3138d56c78564a2194145e9a513832f0',
 'dfa89adfc0cb400195b5f98d6cb9b05c',
 'f12a8834338f45719c8ddce5ab9c48bf',
 '967e846771544605800719d2c74026a1',
 'a04681cbf6ed4f0d964146b20d8c79ad',
 '591d2bda03d5411a8bb388e55487fcfc',
 '837e4ba2e2324cdb9c12eb96f1bc7613',
 '2d614208225e4e5dae8d4f98e4361288',
 '06ccf0d3127a40d5928a1451444d61e4',
 'af48a9d50ab84406a087906a3d672cc3',
 

In [50]:
te3_retriever = qdrant_vector_store.as_retriever()

In [51]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

In [52]:
from langchain.chains import create_retrieval_chain

te3_retrieval_chain = create_retrieval_chain(te3_retriever, document_chain)

In [53]:
answers = []
contexts = []

for question in test_questions:
  response = te3_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [54]:
te3_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [55]:
te3_advanced_retrieval_results = evaluate(te3_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [57]:
te3_advanced_retrieval_results

{'faithfulness': 0.8338, 'answer_relevancy': 0.7684, 'context_recall': 0.7000, 'context_precision': 0.7458, 'answer_correctness': 0.6128}

In [58]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['Baseline -> TE3'] = df_merged['TE3'] - df_merged['ADA']

df_merged

,Metric,ADA,TE3,Baseline -> TE3
0,faithfulness,0.670853,0.833770,0.162917
1,answer_relevancy,0.769766,0.768356,-0.001410
2,context_recall,0.750000,0.700000,-0.050000
3,context_precision,0.697222,0.745833,0.048611
4,answer_correctness,0.646875,0.612831,-0.034044


####❓ Question #3:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## Task 5: Selecting an Advanced Retriever and Evaluating

#### 🏗️ Activity #2

While the changes that occured due to modifying the embedding model were desirable - you're now tasked with improving `context_recall`, or `context_precision` (or both!).

You'll follow these steps:

1. Reason about this list of Advanced Retrieval methods:
  - [Contextual Compression (Reranker)](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/)
  - [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)
  - [Parent Document Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/)
2. Select the method you think will be the most performant.
3. Implement that method.
4. Create a LCEL chain that utlizes the new Retriever method.
5. Evaluate this LCEL and compare to the TE3 results.

> NOTE: We will spend more time in Session 14 diving into advanced retrieval methods, this activity is only to serve as a basic introduction to the idea of component-wise improvements and how they might impact metrics.

In [ ]:
### YOUR CODE HERE

#### 🚧 BONUS CHALLENGE 🚧

> NOTE: Completing this challenge will provide full marks on the assignment, regardless of the complete of the notebook. You do not need to complete this in the notebook for full marks.

##### **MINIMUM REQUIREMENTS**:

1. Baseline `LCEL RAG` Application using `NAIVE RETRIEVAL`
2. Baseline Evaluation using `RAGAS METRICS`
  - [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
  - [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
  - [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
  - [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
  - [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)
3. Implement a `SEMANTIC CHUNKING STRATEGY`.
4. Create an `LCEL RAG` Application using `SEMANTIC CHUNKING` with `NAIVE RETRIEVAL`.
5. Compare and contrast results.

##### **SEMANTIC CHUNKING REQUIREMENTS**:

Chunk semantically similar (based on designed threshold) sentences, and then paragraphs, greedily, up to a maximum chunk size. Minimum chunk size is a single sentence.

Have fun!